## Load ANME-SRB model and identify organism-specific reactions

Load the combined ANME-SRB model and identify which reactions belong to ANME vs SRB
based on the reaction ID prefix (ANME_ or SRB_).

**Input**: models/anme_srb_annotated.json
**Output**: Analysis of reaction distribution between organisms

In [ ]:
%run util.py

# Load the combined model using MSModelUtil
mdlutl = MSModelUtil.from_cobrapy('models/anme_srb_tmfa.json')
model = mdlutl.model

print(f"Loaded model: {model.id}")
print(f"  Metabolites: {len(model.metabolites)}")
print(f"  Reactions: {len(model.reactions)}")
print(f"  Compartments: {list(model.compartments.keys())}")

# Identify ANME and SRB reactions based on ID prefix
anme_rxns = [rxn for rxn in model.reactions if rxn.id.startswith('ANME_')]
srb_rxns = [rxn for rxn in model.reactions if rxn.id.startswith('SRB_')]
other_rxns = [rxn for rxn in model.reactions if not rxn.id.startswith('ANME_') and not rxn.id.startswith('SRB_')]

print(f"\nReaction distribution:")
print(f"  ANME reactions: {len(anme_rxns)}")
print(f"  SRB reactions: {len(srb_rxns)}")
print(f"  Other reactions: {len(other_rxns)}")

print(f"\nANME reactions:")
for rxn in anme_rxns:
    print(f"  {rxn.id}: {rxn.name}")

print(f"\nSRB reactions:")
for rxn in srb_rxns:
    print(f"  {rxn.id}: {rxn.name}")

if other_rxns:
    print(f"\nOther reactions:")
    for rxn in other_rxns:
        print(f"  {rxn.id}: {rxn.name}")

# Save reaction lists for reference
util.save('anme_rxn_ids', [rxn.id for rxn in anme_rxns])
util.save('srb_rxn_ids', [rxn.id for rxn in srb_rxns])

## Create separate ANME model

Create a new model containing only ANME reactions, then remove any orphan metabolites
that are not involved in any reactions. Convert metabolite IDs from bracket notation
(e.g., `h[c]`) to underscore notation (e.g., `h_c`) for better MSCommunity compatibility.
Add exchange reactions for any extracellular metabolites that don't already have them.

**Input**: Original combined model
**Output**: Saves ANME-only model to models/anme_only.json

In [1]:
%run util.py
import re

# Load the combined model
mdlutl = MSModelUtil.from_cobrapy('models/anme_srb_tmfa.json')
model = mdlutl.model

# Create a copy for the ANME model
anme_model = model.copy()
anme_model.id = "ANME"
anme_model.name = "ANME Archaeon Model"

# Remove SRB reactions
srb_rxns_to_remove = [rxn for rxn in anme_model.reactions if rxn.id.startswith('SRB_')]
anme_model.remove_reactions(srb_rxns_to_remove, remove_orphans=False)
print(f"Removed {len(srb_rxns_to_remove)} SRB reactions")

# Find and remove orphan metabolites (not involved in any reactions)
orphan_mets = []
for met in anme_model.metabolites:
    if len(met.reactions) == 0:
        orphan_mets.append(met)

print(f"\nFound {len(orphan_mets)} orphan metabolites:")
for met in orphan_mets:
    print(f"  {met.id}: {met.name}")

# Remove orphan metabolites
anme_model.remove_metabolites(orphan_mets)
print(f"\nRemoved {len(orphan_mets)} orphan metabolites")

# Convert metabolite IDs from [compartment] to _compartment notation
print(f"\nConverting metabolite ID compartment notation...")
renamed_count = 0
for met in list(anme_model.metabolites):
    # Match pattern like "met_id[c]" and convert to "met_id_c"
    match = re.match(r'^(.+)\[([a-zA-Z0-9]+)\]$', met.id)
    if match:
        base_id = match.group(1)
        compartment = match.group(2)
        new_id = f"{base_id}_{compartment}"
        met.id = new_id
        renamed_count += 1

print(f"Renamed {renamed_count} metabolites to underscore notation")

# Wrap in MSModelUtil for adding exchanges
anme_mdlutl = MSModelUtil.get(anme_model)

# Add exchange reactions for extracellular metabolites
extracellular_compartments = ['e', 'e0', 'env']
extracellular_mets = [met for met in anme_model.metabolites 
                      if met.compartment in extracellular_compartments]

if extracellular_mets:
    # Check which already have exchanges
    existing_exchanges = anme_mdlutl.exchange_hash()
    mets_needing_exchanges = [met for met in extracellular_mets 
                              if met not in existing_exchanges]
    
    if mets_needing_exchanges:
        anme_mdlutl.add_exchanges_for_metabolites(
            mets_needing_exchanges, 
            uptake=1000, 
            excretion=1000
        )
        print(f"\nAdded exchange reactions for {len(mets_needing_exchanges)} extracellular metabolites:")
        for met in mets_needing_exchanges:
            print(f"  {met.id}: {met.name}")
    else:
        print(f"\nAll {len(extracellular_mets)} extracellular metabolites already have exchanges")
else:
    print(f"\nNo extracellular metabolites found")

# Summary
print(f"\nANME model summary:")
print(f"  Reactions: {len(anme_model.reactions)}")
print(f"  Metabolites: {len(anme_model.metabolites)}")
print(f"  Compartments: {list(anme_model.compartments.keys())}")
print(f"  Exchange reactions: {len(anme_mdlutl.exchange_list())}")

# Show sample metabolite IDs
print(f"\nSample metabolite IDs:")
for met in list(anme_model.metabolites)[:5]:
    print(f"  {met.id}")

# Save model
anme_mdlutl.save_model('models/anme_only.json')
print(f"\nSaved ANME model to models/anme_only.json")

# Save metabolite list for reference
util.save('anme_metabolites', [met.id for met in anme_model.metabolites])

/Users/chenry/Dropbox/Projects/KBUtilLib/src
modelseedpy 0.4.2


2025-12-30 01:02:09,812 - __main__.NotebookUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2025-12-30 01:02:09,813 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-12-30 01:02:09,813 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token


cobrakbase 0.4.0
loading biochemistry database from /Users/chenry/Dropbox/Projects/ModelSEEDDatabase


2025-12-30 01:02:14,724 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/ModelSEEDDatabase
2025-12-30 01:02:15,112 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-12-30 01:02:15,114 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-12-30 01:02:15,115 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


/Users/chenry/.npm-global/bin/claude


2025-12-30 01:02:15,618 - __main__.NotebookUtil - INFO - AICurationUtils initialized with backend: claude-code


Removed 10 SRB reactions

Found 13 orphan metabolites:
  h[m]: proton on the outside of the membrane
  na[m]: Sodium cation on the outside of the membrane
  amp[c]: AMP
  aps[c]: adenosine-5′-phosphosulfate
  hs[m]: sulfide outside of the membrane
  mqox[m]: menequinone oxidized 
  mqred[m]: menequinone reduced 
  ppi[c]: Pyrophosphate
  so3[c]: Sulfite
  so4[c]: sulfate in the cytoplasm
  so4[m]: sulfate outside of the membrane
  tpicox[c]: type I cytochrome c3 oxidized
  tpicred[c]: type I cytochrome c3 reduced

Removed 13 orphan metabolites

Converting metabolite ID compartment notation...
Renamed 31 metabolites to underscore notation

All 6 extracellular metabolites already have exchanges

ANME model summary:
  Reactions: 20
  Metabolites: 31
  Compartments: ['c', 'e', 'm']
  Exchange reactions: 6

Sample metabolite IDs:
  adp_c
  atp_c
  ch4_c
  ch4_e
  co2_c

Saved ANME model to models/anme_only.json


## Create separate SRB model

Create a new model containing only SRB reactions, then remove any orphan metabolites
that are not involved in any reactions. Convert metabolite IDs from bracket notation
(e.g., `h[c]`) to underscore notation (e.g., `h_c`) for better MSCommunity compatibility.
Add exchange reactions for any extracellular metabolites that don't already have them.

**Input**: Original combined model
**Output**: Saves SRB-only model to models/srb_only.json

In [2]:
%run util.py
import re

# Load the combined model
mdlutl = MSModelUtil.from_cobrapy('models/anme_srb_tmfa.json')
model = mdlutl.model

# Create a copy for the SRB model
srb_model = model.copy()
srb_model.id = "SRB"
srb_model.name = "Sulfate-Reducing Bacterium Model"

# Remove ANME reactions
anme_rxns_to_remove = [rxn for rxn in srb_model.reactions if rxn.id.startswith('ANME_')]
srb_model.remove_reactions(anme_rxns_to_remove, remove_orphans=False)
print(f"Removed {len(anme_rxns_to_remove)} ANME reactions")

# Find and remove orphan metabolites (not involved in any reactions)
orphan_mets = []
for met in srb_model.metabolites:
    if len(met.reactions) == 0:
        orphan_mets.append(met)

print(f"\nFound {len(orphan_mets)} orphan metabolites:")
for met in orphan_mets:
    print(f"  {met.id}: {met.name}")

# Remove orphan metabolites
srb_model.remove_metabolites(orphan_mets)
print(f"\nRemoved {len(orphan_mets)} orphan metabolites")

# Convert metabolite IDs from [compartment] to _compartment notation
print(f"\nConverting metabolite ID compartment notation...")
renamed_count = 0
for met in list(srb_model.metabolites):
    # Match pattern like "met_id[c]" and convert to "met_id_c"
    match = re.match(r'^(.+)\[([a-zA-Z0-9]+)\]$', met.id)
    if match:
        base_id = match.group(1)
        compartment = match.group(2)
        new_id = f"{base_id}_{compartment}"
        met.id = new_id
        renamed_count += 1

print(f"Renamed {renamed_count} metabolites to underscore notation")

# Wrap in MSModelUtil for adding exchanges
srb_mdlutl = MSModelUtil.get(srb_model)

# Add exchange reactions for extracellular metabolites
extracellular_compartments = ['e', 'e0', 'env']
extracellular_mets = [met for met in srb_model.metabolites 
                      if met.compartment in extracellular_compartments]

if extracellular_mets:
    # Check which already have exchanges
    existing_exchanges = srb_mdlutl.exchange_hash()
    mets_needing_exchanges = [met for met in extracellular_mets 
                              if met not in existing_exchanges]
    
    if mets_needing_exchanges:
        srb_mdlutl.add_exchanges_for_metabolites(
            mets_needing_exchanges, 
            uptake=1000, 
            excretion=1000
        )
        print(f"\nAdded exchange reactions for {len(mets_needing_exchanges)} extracellular metabolites:")
        for met in mets_needing_exchanges:
            print(f"  {met.id}: {met.name}")
    else:
        print(f"\nAll {len(extracellular_mets)} extracellular metabolites already have exchanges")
else:
    print(f"\nNo extracellular metabolites found")

# Summary
print(f"\nSRB model summary:")
print(f"  Reactions: {len(srb_model.reactions)}")
print(f"  Metabolites: {len(srb_model.metabolites)}")
print(f"  Compartments: {list(srb_model.compartments.keys())}")
print(f"  Exchange reactions: {len(srb_mdlutl.exchange_list())}")

# Show sample metabolite IDs
print(f"\nSample metabolite IDs:")
for met in list(srb_model.metabolites)[:5]:
    print(f"  {met.id}")

# Save model
srb_mdlutl.save_model('models/srb_only.json')
print(f"\nSaved SRB model to models/srb_only.json")

# Save metabolite list for reference
util.save('srb_metabolites', [met.id for met in srb_model.metabolites])

2025-12-30 01:02:28,554 - __main__.NotebookUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2025-12-30 01:02:28,555 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-12-30 01:02:28,556 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-12-30 01:02:28,557 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/ModelSEEDDatabase


/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-12-30 01:02:28,941 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-12-30 01:02:28,943 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-12-30 01:02:28,943 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


/Users/chenry/.npm-global/bin/claude


2025-12-30 01:02:29,416 - __main__.NotebookUtil - INFO - AICurationUtils initialized with backend: claude-code


Removed 14 ANME reactions

Found 21 orphan metabolites:
  ch4[c]: methane
  co2[c]: CO2
  cob[c]: coenzyme-b
  com[c]: coenzyme-m
  f420-2[c]: coenzyme-ferredoxin-420-2-oxidized
  f420-2h2[c]: coenzyme-ferredoxin-420-2-reduced
  fdox[c]: ferredoxin-oxidized-24Fe-4S
  fdred[c]: ferredoxin-reduced-24Fe-4S
  formh4spt[c]: formyltetrahydrosarcinapterin
  formmfr(b)[c]: formylmethanofuran-b
  h4spt[c]: tetrahydrosarcinapterin
  hsfd[c]: heterodisulfide
  mcom[c]: methylcoenzyme-m
  menylh4spt[c]: methenyl-tetrahydrosarcinapterin
  mfr(b)[c]: methanofuran-b
  mh4spt[c]: N5-methyl-tetrahydrosarcinapterin
  mleneh4spt[c]: N5-N10-methylene-5-6-7-8-tetrahydrosarcinapterin
  mphen[m]: methanophenazine-oxidized
  mphenh2[m]: methanophenazine-reduced
  na1[c]: Sodium in the cytoplasm
  na[m]: Sodium cation on the outside of the membrane

Removed 21 orphan metabolites

Converting metabolite ID compartment notation...
Renamed 23 metabolites to underscore notation

All 6 extracellular metabolites alre

## Combine models using MSCommunity

Use MSCommunity.build_from_species_models() to properly combine the ANME and SRB
models into a community model with separate compartments for each organism.

**Input**: models/anme_only.json, models/srb_only.json
**Output**: Saves community model to models/anme_srb_community.json

In [3]:
%run util.py
from modelseedpy.community import MSCommunity

# Load the individual models
anme_mdlutl = MSModelUtil.from_cobrapy('models/anme_only.json')
srb_mdlutl = MSModelUtil.from_cobrapy('models/srb_only.json')

anme_model = anme_mdlutl.model
srb_model = srb_mdlutl.model

print(f"ANME model: {len(anme_model.reactions)} reactions, {len(anme_model.metabolites)} metabolites")
print(f"SRB model: {len(srb_model.reactions)} reactions, {len(srb_model.metabolites)} metabolites")

# Build community model using MSCommunity
print(f"\nBuilding community model...")
community = MSCommunity.build_from_species_models(
    models=[anme_model, srb_model],
    mdlid="ANME_SRB_Community",
    name="ANME-SRB Syntrophic Community",
    names=["ANME", "SRB"],
    abundances={"ANME": 0.5, "SRB": 0.5}  # Equal abundances initially
)

# Get the community model
community_model = community.model

print(f"\nCommunity model created:")
print(f"  ID: {community_model.id}")
print(f"  Reactions: {len(community_model.reactions)}")
print(f"  Metabolites: {len(community_model.metabolites)}")
print(f"  Compartments: {list(community_model.compartments.keys())}")

# Show species in community
print(f"\nSpecies in community:")
for species in community.species:
    print(f"  {species.id}: {species.name}")

# Save the community model
community_mdlutl = MSModelUtil.get(community_model)
community_mdlutl.save_model('models/anme_srb_community.json')
print(f"\nSaved community model to models/anme_srb_community.json")

# Save community info
util.save('community_info', {
    'model_id': community_model.id,
    'reactions': len(community_model.reactions),
    'metabolites': len(community_model.metabolites),
    'compartments': list(community_model.compartments.keys()),
    'species': [s.id for s in community.species]
})

2025-12-30 01:02:45,167 - __main__.NotebookUtil - INFO - Loaded configuration from: /Users/chenry/.kbutillib/config.yaml
2025-12-30 01:02:45,167 - __main__.NotebookUtil - INFO - Loaded 0 tokens from /Users/chenry/.tokens
2025-12-30 01:02:45,168 - __main__.NotebookUtil - INFO - Loaded kbase tokens from /Users/chenry/.kbase/token
2025-12-30 01:02:45,169 - __main__.NotebookUtil - INFO - ModelSEED database loaded from /Users/chenry/Dropbox/Projects/ModelSEEDDatabase


/Users/chenry/Dropbox/Projects/KBUtilLib/src


2025-12-30 01:02:45,574 - __main__.NotebookUtil - WARNING - BLAST tools not found. Install NCBI BLAST+ to use BLAST functionality. On Ubuntu/Debian: sudo apt-get install ncbi-blast+, On MacOS: brew install blast
2025-12-30 01:02:45,575 - __main__.NotebookUtil - INFO - Notebook environment detected
2025-12-30 01:02:45,575 - __main__.NotebookUtil - INFO - ArgoGatewayClient initialised | model=gpto3mini env=dev timeout=120.0s url=https://apps-dev.inside.anl.gov/argoapi/api/v1/resource/streamchat/


/Users/chenry/.npm-global/bin/claude


2025-12-30 01:02:46,055 - __main__.NotebookUtil - INFO - AICurationUtils initialized with backend: claude-code
CRITICAL:modelseedpy.community.mscommunity:Primary biomass reaction not found in community model


ANME model: 20 reactions, 31 metabolites
SRB model: 16 reactions, 23 metabolites

Building community model...
bio1 {<Metabolite cpd11416_c0 at 0x3593a8b20>: 1}
SK_cpd11416_c0 {<Metabolite cpd11416_c0 at 0x3593a8b20>: -1}

Community model created:
  ID: ANME_SRB_Community
  Reactions: 32
  Metabolites: 49
  Compartments: ['c1', 'e0', 'm1', 'm2', 'c2', 'c0']

Species in community:

Saved community model to models/anme_srb_community.json


## Verify community model structure

Examine the community model to verify that reactions and metabolites are properly
compartmentalized for each organism.

**Input**: models/anme_srb_community.json
**Output**: Summary of model structure

In [ ]:
%run util.py

# Load the community model
community_mdlutl = MSModelUtil.from_cobrapy('models/anme_srb_community.json')
model = community_mdlutl.model

print(f"Community model: {model.id}")
print(f"  Total reactions: {len(model.reactions)}")
print(f"  Total metabolites: {len(model.metabolites)}")

# Analyze compartments
print(f"\nCompartments:")
for cmp_id, cmp_name in model.compartments.items():
    mets_in_cmp = [m for m in model.metabolites if m.compartment == cmp_id]
    print(f"  {cmp_id} ({cmp_name}): {len(mets_in_cmp)} metabolites")

# Analyze reactions by compartment pattern
print(f"\nReaction distribution:")
rxn_patterns = {}
for rxn in model.reactions:
    compartments = set()
    for met in rxn.metabolites:
        compartments.add(met.compartment)
    pattern = ','.join(sorted(compartments))
    if pattern not in rxn_patterns:
        rxn_patterns[pattern] = []
    rxn_patterns[pattern].append(rxn.id)

for pattern, rxns in sorted(rxn_patterns.items()):
    print(f"  Compartments [{pattern}]: {len(rxns)} reactions")

# Show exchange reactions
exchanges = community_mdlutl.exchange_list()
print(f"\nExchange reactions: {len(exchanges)}")
for ex in exchanges[:10]:  # Show first 10
    print(f"  {ex.id}: {ex.reaction}")
if len(exchanges) > 10:
    print(f"  ... and {len(exchanges) - 10} more")

## Test community model FBA

Run a simple FBA on the community model to verify it can carry flux.

**Input**: models/anme_srb_community.json
**Output**: FBA results

In [ ]:
%run util.py
from modelseedpy.community import MSCommunity

# Load the community model
community_mdlutl = MSModelUtil.from_cobrapy('models/anme_srb_community.json')
model = community_mdlutl.model

# Make all reactions reversible for testing
for rxn in model.reactions:
    rxn.lower_bound = -1000
    rxn.upper_bound = 1000

# Find a reaction to use as objective
# Look for methane-related reactions (ANME pathway entry)
ch4_rxns = [rxn for rxn in model.reactions if 'ch4' in rxn.id.lower() or 'methane' in rxn.name.lower()]
print(f"Methane-related reactions:")
for rxn in ch4_rxns:
    print(f"  {rxn.id}: {rxn.name}")
    print(f"    {rxn.reaction}")

# Try to run FBA
if ch4_rxns:
    test_rxn = ch4_rxns[0]
    model.objective = test_rxn.id
    print(f"\nObjective: {test_rxn.id}")
    
    solution = model.optimize()
    print(f"\nFBA Solution:")
    print(f"  Status: {solution.status}")
    print(f"  Objective value: {solution.objective_value:.4f}")
    
    # Show non-zero fluxes
    nonzero_fluxes = {rxn_id: flux for rxn_id, flux in solution.fluxes.items() if abs(flux) > 1e-6}
    print(f"\nNon-zero fluxes: {len(nonzero_fluxes)}")
    for rxn_id, flux in sorted(nonzero_fluxes.items(), key=lambda x: abs(x[1]), reverse=True)[:15]:
        print(f"  {rxn_id}: {flux:.4f}")
else:
    print("No methane-related reactions found")